In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '111_ker_'

### Data Load

In [11]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_hist = utils.read_df_pkl('../input/histori*')
df_new = utils.read_df_pkl('../input/new_mer*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00, 72.08it/s]


SystemExit: 

In [6]:
df_hist_trans = df_hist
df_new_merchant_trans = df_new

In [7]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [8]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [9]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [12]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
# aggs['purchase_amount'] = ['max','mean']
aggs['installments'] = ['max','min','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','mean']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

203

In [13]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['max','min','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','mean']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_purchase_date_diff'] = (df_hist_trans_group['new_purchase_date_max'] - df_hist_trans_group['new_purchase_date_min']).dt.days
df_hist_trans_group['new_purchase_date_average'] = df_hist_trans_group['new_purchase_date_diff']/df_hist_trans_group['new_card_id_size']
df_hist_trans_group['new_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()

28

In [14]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)

,card_id,first_active_month,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_max,hist_installments_min,hist_installments_mean,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_mean,hist_month_diff_mean,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,new_month_nunique,new_hour_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_purchase_amount_var,new_installments_max,new_installments_min,new_installments_mean,new_purchase_date_max,new_purchase_date_min,new_month_lag_max,new_month_lag_mean,new_month_diff_mean,new_authorized_flag_sum,new_authorized_flag_mean,new_weekend_sum,new_weekend_mean,new_category_1_sum,new_category_1_mean,new_card_id_size,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,9,23,35,7,21,94,41,-165.968735,2.258394,-0.739395,-0.638341,0.045003,1,0,0.015385,2018-02-25 09:31:15,2017-06-27 14:18:08,0,-3.911538,11.119231,247,0.950000,90,0.346154,0,0.000000,260,0.072502,0.346719,242,0.930769,336,2.0,8.0,7.0,7.0,10.0,23.0,14.0,-13.242188,-0.296143,-0.724609,-0.575684,0.018433,0.0,0.0,0.00000,2018-04-29 11:23:05,2018-03-05 14:04:36,2.0,1.478261,11.000000,23.0,1.0,6.0,0.260870,0.0,0.000000,23.0,-0.569336,-0.630859,54.0,2.347826,273.0
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,12,24,50,7,24,142,57,-210.006332,4.630300,-0.742400,-0.600018,0.148200,10,-1,1.551429,2018-01-31 22:31:09,2017-01-06 16:29:42,0,-5.031429,12.131429,339,0.968571,132,0.377143,31,0.088571,350,0.074568,-0.295163,390,1.114286,360,2.0,5.0,4.0,4.0,4.0,6.0,5.0,-4.355469,-0.701660,-0.739258,-0.726074,0.000207,1.0,1.0,1.00000,2018-03-30 06:48:26,2018-02-01 17:07:54,2.0,1.500000,12.000000,6.0,1.0,0.0,0.000000,0.0,0.000000,6.0,-0.569336,-0.606445,56.0,9.333333,303.0
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,10,14,22,7,7,13,8,-29.167391,-0.145847,-0.730138,-0.678311,0.007635,0,0,0.000000,2018-02-27 19:08:25,2017-01-11 08:21:22,0,-8.604651,11.093023,41,0.953488,11,0.255814,0,0.000000,43,-0.087803,0.358458,412,9.581395,333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.700195,-0.700195,-0.700195,-0.700195,NaN,0.0,0.0,0.00000,2018-04-28 17:43:11,2018-04-28 17:43:11,2.0,2.000000,11.000000,1.0,1.0,1.0,1.000000,0.0,0.000000,1.0,-0.548828,-0.630859,0.0,0.000000,273.0
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,6,16,20,7,13,50,25,-49.491364,1.445596,-0.740897,-0.642745,0.068447,3,-1,1.090909,2018-02-28 11:44:40,2017-09-26 16:22:21,0,-2.831169,11.064935,77,1.000000,11,0.142857,12,0.155844,77,-0.086166,-0.338321,154,2.000000,333,2.0,5.0,5.0,4.0,5.0,7.0,6.0,-4.656250,-0.566895,-0.734375,-0.665039,0.004345,1.0,1.0,1.00000,2018-04-18 11:00:11,2018-03-07 11:55:06,2.0,1.714286,11.000000,7.0,1.0,3.0,0.428571,1.0,0.142857,7.0,-0.530762,-0.515137,41.0,5.857143,284.0
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,4,22,17,7,17,66,26,-48.687656,7.193041,-0.746156,-0.366073,1.828159,12,1,1.368421,2018-02-28 20:40:41,2017-11-12 00:00:00,0,-1.285714,11.067669,128,0.962406,42,0.315789,15,0.112782,133,-0.114647,-0.377684,108,0.812030,332,2.0,14.0,8.0,7.0,10.0,36.0,17.0,-19.921875,0.450928,-0.739258,-0.553711,0.050110,2.0,1.0,1.02832,2018-04-28 18:50:25,2018-03-02 11:55:43,2.0,1.555556,11.166667,36.0,1.0,12.0,0.333333,2.0,0.055556,36.0,-0.546387,-0.570801,57.0,1.583333

In [15]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [16]:
for df in [df_train, df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['new_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_purchase_date_max',\
                     'new_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_card_id_size'].fillna(0)+df['hist_card_id_size'].fillna(0)
    df['purchase_amount_total'] = df['new_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)

In [18]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']
del df_train['target']

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": -1,
         "random_state": seed}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [33]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("../submit/1213_kernel_LB3699_submission_postpro_less_than_-10.csv", index=False)

In [19]:
sub_df = pd.read_csv('../submit/1213_082_submit_lgb_rate0.01_84features_CV3.6548233901685974_LB.csv')
print(sub_df['target'][sub_df['target']<-12].shape)
# sub_df['target'][sub_df['target']<-12] = -33.21928095
sub_df.sort_values(by='target', ascending=True)

SystemExit: 

In [18]:
df_feat['hist_purchase_date_min'].head()

0    1.49857e+09
1    1.48372e+09
2    1.48412e+09
3    1.50644e+09
4    1.51044e+09
Name: hist_purchase_date_min, dtype: object

In [19]:
import sys
sys.path.append('../../data_analysis/library/')
import utils

ignore_features = ['first_active_month', 'card_id', 'outlier', 'target']

df_feat = pd.concat([df_train, df_test], axis=0)

for col in df_train_columns:
    if col in ignore_features:
        continue
    if col.count('purchase_date_min') or col.count('purchase_date_max'):
        continue
#     if not(col.count('feature_')):continue
#     if not(col.count('card_id_total')):continue

    try:
        feature = df_feat[col].astype('float32').values
    except TypeError:
        print(col)
        sys.exit()

    utils.to_pkl_gzip(path = f'../features/1_first_valid/111_ker_{col}@', obj=feature)